In [77]:
#Import libraries
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
print("TF version:", tf.__version__)
print("TF Hub version:", hub.__version__)

#Check whether theres GPU available
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

TF version: 2.9.1
TF Hub version: 0.12.0
GPU not available


In [32]:
IMG_SIZE = 224
batch_size = 32
#Function for preprocessing
def process_image(filename, img_size=IMG_SIZE):
   #read image
   image = tf.io.read_file(filename)

   #turn jpeg to numerical Tensor with 3 color channels (RGB)
   image = tf.image.decode_jpeg(image, channels=3)

   #Convert colour channels values 0-255 to 0-1 values.
   #This is a normalization process to make process more efficient.
   image = tf.image.convert_image_dtype(image, tf.float32)

   #Resize to (224,224)
   image = tf.image.resize(image, size=[img_size, img_size])

   return image

# Function for data configuration (for performance) 
def configure_tensor(ds, shuffle=False):
   if shuffle: # For train set
      ds = ds.shuffle(buffer_size=1000) 
   ds = ds.batch(batch_size)
   ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
   return ds

print(f"Creating test data batches... BATCH SIZE={batch_size}")
x = "./Images/custom-images/WhatsApp Image 2022-06-16 at 8.28.28 PM.jpeg"
x_test = process_image(x)
x_testt = np.array([x_test], dtype='float32')
# x_testt = configure_tensor(x_test)

Creating test data batches... BATCH SIZE=32


In [78]:
image = tf.io.read_file("./Images/custom-images/WhatsApp Image 2022-06-16 at 8.28.28 PM.jpeg")

#turn jpeg to numerical Tensor with 3 color channels (RGB)
image = tf.image.decode_jpeg(image, channels=3)

image = tf.image.convert_image_dtype(image, tf.uint8)

In [80]:
interpreter = tf.lite.Interpreter("./object_model.tflite")
interpreter.allocate_tensors()

In [81]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [83]:
im_height, im_width = image.shape[:2]
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], [1, im_height, im_width, 3])
interpreter.allocate_tensors()

In [185]:
interpreter.set_tensor(input_details[0]['index'], [image])

IndexError: list index out of range

In [86]:
interpreter.invoke()

In [188]:
output_details[7]
# [   1, 1917,    4] raw detect boxes 6 = 0
# 1 = [1,1] [1,-1] raw detect anchor0 = 1
# [1] = NUM = 2
# [   1, 1917,   91] raw detect scores = 3
# [1,1,1] [1,-1,-1] detection boxes = 4
#  = [1,1] [1,-1] detection classes = 5
#  = [1,1] [1,-1] detection scores = 6
# [1,1,1] [1,-1,-1] detection multiclass scores = 7

{'name': 'StatefulPartitionedCall:3',
 'index': 2163,
 'shape': array([1, 1, 1], dtype=int32),
 'shape_signature': array([ 1, -1, -1], dtype=int32),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [186]:
output = interpreter.get_tensor(output_details[7]['index'])

In [187]:
output.shape

(1, 100, 91)

In [184]:
output[0]

array([1857.,  456., 1696., 1472.,  456.,  456., 1211.,  106.,  456.,
        228.,  277.,  456., 1220.,  111.,  106.,  344.,  241.,  234.,
        456., 1698.,  173.,   57., 1698.,  342.,  176., 1840., 1913.,
       1840.,  111., 1859.,   66.,  122.,  347., 1724.,  114.,  285.,
        171.,  456.,  230.,   66.,  456.,  237.,  234.,  234.,  401.,
         57., 1327.,  456.,  399.,  456.,  122., 1116.,  238.,  237.,
        347.,  274.,   68., 1205.,   66., 1207.,  237.,  234.,  233.,
        237.,  228.,  173.,  456.,  185.,   64.,  830., 1880., 1156.,
        238.,  228.,  234.,  119.,  228.,  238.,  345.,  238.,  456.,
        180.,  277., 1698.,  237., 1267.,  228.,  240., 1298.,   67.,
       1122.,  123.,  173.,  239., 1912.,  456., 1724.,  237.,  241.,
        285.], dtype=float32)

In [43]:
labels = []
for folders in os.listdir("./Images/train_images"):
         labels.append("".join(folders.split("-")[1:]))
unique_breeds = np.unique(labels)

In [49]:
with open('classes.txt', mode='w') as file:
   for x in unique_breeds:
      file.writelines(x + '\n')

In [54]:
classes = []
with open("./classes.txt", mode="r") as file:
   for row in file:
      classes.append(row.rstrip("\n"))
print(classes)

['Afghan_hound', 'African_hunting_dog', 'Airedale', 'American_Staffordshire_terrier', 'Appenzeller', 'Australian_terrier', 'Bedlington_terrier', 'Bernese_mountain_dog', 'Blenheim_spaniel', 'Border_collie', 'Border_terrier', 'Boston_bull', 'Bouvier_des_Flandres', 'Brabancon_griffon', 'Brittany_spaniel', 'Cardigan', 'Chesapeake_Bay_retriever', 'Chihuahua', 'Dandie_Dinmont', 'Doberman', 'English_foxhound', 'English_setter', 'English_springer', 'EntleBucher', 'Eskimo_dog', 'French_bulldog', 'German_shepherd', 'German_shorthaired_pointer', 'Gordon_setter', 'Great_Dane', 'Great_Pyrenees', 'Greater_Swiss_Mountain_dog', 'Ibizan_hound', 'Irish_setter', 'Irish_terrier', 'Irish_water_spaniel', 'Irish_wolfhound', 'Italian_greyhound', 'Japanese_spaniel', 'Kerry_blue_terrier', 'Labrador_retriever', 'Lakeland_terrier', 'Leonberg', 'Lhasa', 'Maltese_dog', 'Mexican_hairless', 'Newfoundland', 'Norfolk_terrier', 'Norwegian_elkhound', 'Norwich_terrier', 'Old_English_sheepdog', 'Pekinese', 'Pembroke', 'Pom

In [55]:
len(classes)

120